In [1]:
import pandas as pd
import re
import os
import spacy
from scipy import constants
from spacy_conll import init_parser
from nltk.tokenize import word_tokenize
from nltk.translate import AlignedSent
from nltk.translate import IBMModel1
from langdetect import detect
from googletrans import Translator
from tqdm import tqdm
import pandas as pd
from spacy.matcher import PhraseMatcher
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import textwrap
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, LancasterStemmer

In [3]:
# DATA_PATH = '<path for your resume/job description file>'
DATA_PATH = './input/arrayJobList.json'
# SKILL_FILE_PATH = '<path for the skill-set file>'
SKILL_FILE_PATH = './input/Updated_Skillset_Data_Lowercase.xlsx'

# Matcher the skill from the corpus with the skill extracted

In [4]:
def init_parser_func(nlp, SKILL_FILE_PATH, file_type='csv'):
    """
    This function initializes the nlp PhraseMatcher
    and reads the skill-set data file in an excel/csv format
    
    :param nlp: It is an NLP load instance, ex: nlp.load("en_core_web_sm")
    :param SKILL_FILE_PATH: Path for the data file
    :param file_type: csv/excel, default = csv
    :return: Returns a PhraseMatcher object.
    """
    
    if file_type == "excel":
        keyword_df = pd.read_excel(SKILL_FILE_PATH)
    elif file_type == "csv":
        keyword_df = pd.read_csv(SKILL_FILE_PATH)
    else:
        raise ValueError("Unsupported file type. Supported types: 'excel', 'csv'.")
    
    matcher = PhraseMatcher(nlp.vocab)
    
    for column in keyword_df.columns:
        patterns = [nlp(text) for text in keyword_df[column].dropna()]
        matcher.add(column, None, *patterns)
    
    return matcher


In [5]:
# def create_profile(nlp, matcher,text,application_subject):
# 	"""
# 	This funciton creates a profile of one sample row 
# 	in the dataset

# 	:param nlp: nlp load object with phrasematcher already initialized
# 	:param matcher: Custom PhraseMatcher object
# 	:param text: Resume/JD text data
# 	:param application_subject: company/applicant's name 
# 	:return: returns the particular profile in a dataframe format
# 	"""	

# 	# Get the matches from the text
# 	doc = nlp(text)
# 	matches = matcher(doc)
	
# 	# Create a dataframe to return
# 	d = []
# 	for match_id, start, end in matches:
# 		rule_id = nlp.vocab.strings[match_id]  # get the Skill, eg: 'Machine Learning'
# 		span = doc[start : end]  # get the Sub-skill, eg: 'Regression'
# 		d.append((rule_id, span.text))
# 	data = []
# 	for each,count in Counter(d).items():
# 		data.append([application_subject,*each,count])
# 	dataf = pd.DataFrame(data,columns=['Company/Candidate Name','Skill','Sub-skill','Count'])
# 	return(dataf)

def create_profile(nlp, matcher, text, role, application_subject):
    """
    This function creates a profile of one sample row 
    in the dataset

    :param nlp: nlp load object with phrasematcher already initialized
    :param matcher: Custom PhraseMatcher object
    :param text: Resume/JD text data
    :param application_subject: company/applicant's name 
    :return: returns the particular profile in a dataframe format
    """    

    # Get the matches from the text
    doc = nlp(text)
    matches = matcher(doc)
    
    # Create a list to store extracted skills
    skills = []

    # Iterate through matches and extract skills
    for match_id, start, end in matches:
        rule_id = nlp.vocab.strings[match_id]  # get the Skill, eg: 'Machine Learning'
        span = doc[start : end]  # get the Sub-skill, eg: 'Regression'
        skills.append((application_subject, rule_id, span.text))
    
    # Count occurrences of each skill
    skill_counts = Counter(skills)
    
    # Prepare data for DataFrame
    data = []
    for (company_name, skill, sub_skill), count in skill_counts.items():
        data.append([company_name, role, skill, sub_skill, count])
    
    # Create DataFrame
    dataf = pd.DataFrame(data, columns=['Company/Candidate Name', 'Role', 'Skill', 'Sub-skill', 'Count'])
    
    return dataf


def plot_df(final_database):
	"""
	This function simply plots the final
	plot for skill analysis
	And also saves the final png file

	:param final_database: dataframe of employees/JD and skills
	:output 1: saves a png file
	:output 2: saves a final excel sheet 
	"""

	final_database2 = final_database['Sub-skill'].groupby([final_database['Company/Candidate Name'], final_database['Skill']]).count().unstack()
	final_database2.reset_index(inplace = True)
	final_database2.fillna(0,inplace=True)
	new_data = final_database2.iloc[:,1:]
	new_data.index = final_database2['Company/Candidate Name']
	
	# Execute the below line if you want to see the JD profile in a csv format
	new_data.to_csv('./output/skillset.csv')
	
	plt.rcParams.update({'font.size': 6})
	ax = new_data.plot.barh(title="JD/Resume keywords by category", legend=True, figsize=(26,26), stacked=True)
	labels = []
	for j in new_data.columns:
		for i in new_data.index:
			label = str(j)+": " + str(new_data.loc[i][j])
			labels.append(label)
	patches = ax.patches
	for label, rect in zip(labels, patches):
		width = rect.get_width()
		if width > 0:
			x = rect.get_x()
			y = rect.get_y()
			height = rect.get_height()
			ax.text(x + width/2., y + height/2., label, ha='center', va='center')
	plt.savefig('./output/graph.png')
	plt.show()


# Data Preprocessing

In [6]:
def wrap_text(text, width=150):
    return '\n'.join(textwrap.wrap(text, width=width))

def clean_text(text):
    if isinstance(text, str):  # Check if text is a string
        # Remove extra whitespace characters and newline characters
        text = re.sub(r'\s+', ' ', text)
        
        # Replace specific unwanted characters (customize as needed)
        text = text.replace('：', ':')
        
        # Remove digits
        text = re.sub(r'\d+', '', text)
        
        # Remove non-ASCII characters (if necessary)
        text = ''.join(char for char in text if 31 < ord(char) < 128)
        
        return text.strip()
    else:
        return ""

def translate_text(text):
    try:
        # Detect the language of the input text
        lang = detect(text)
        
        # If the detected language is English, return the original text
        if lang == 'en':
            return text
    
        # Initialize the Translator object
        translator = Translator()
        
        # Translate text from detected language to English
        translation = translator.translate(text, src=lang, dest='en')
        
        # Return the translated text
        return translation.text
    
    except Exception as e:
        print(f"Translation error: {e}")
        return None
    
def join_tokens(tokens):
    if isinstance(tokens, list):  # Check if tokens are already a list of tokens
        return ' '.join(tokens)   # Join tokens into a sentence
    else:
        return ''  

In [7]:
# Read the Excel file
df = pd.read_json(DATA_PATH)
df.head(5)

,Company Name,Role,Location,Posted Time,Applicants,Work Type,Employment Type,Company Size,About The Job,About The Company
0,Entravision APAC,Client Partner - Pinterest,"Ho Chi Minh City, Vietnam",2 weeks ago,40 applicants,Hybrid,Full-time,51-200,"Entravision is a leading advertising, media, c...",Entravision is an online advertising and techn...
1,Ivoclar,Business Development Manager,"Ho Chi Minh City, Vietnam",2 weeks ago,Over 100 applicants,On-site,Full-time,"1,001-5,000",Your Responsibilities Implementing Ivocla...,Going strong since 1923! About The Ivoclar Gro...
2,Hyphen Deux Co. Ltd.,RTL DESIGN ENGINEER,"Thủ Đức, Ho Chi Minh City, Vietnam",2 days ago,16 applicants,On-site,Full-time,11-50,"We have exciting job openings, including the p...",Let’s step into the era of Cutting-Edge AI Chi...
3,SID Global Solutions,Enterprise Sales Executive,"Ho Chi Minh City, Vietnam",2 weeks ago,29 applicants,On-site,Full-time,201-500,Role: Enterprise Sales Executive (BFSI Domain)...,SID Global Solutions is a premium implementati...
4,PHILIA DX,UNREAL ENGINE ARTIST / CG Generalist,"Ho Chi Minh City, Vietnam",6 days ago,4 applicants,On-site,Full-time,51-200,PHILIA DX RECRUIMENT - UNREAL ENGINE ARTIST(Se...,PHILIA DX IS A COMPANY THAT THRIVES AT THE INT...


In [8]:
df = df[["Company Name","Role","About The Job"]]

In [9]:
df.shape

(1000, 3)

In [10]:
df.isnull().sum()

Company Name     67
Role              0
About The Job     0
dtype: int64

In [11]:
df.dropna(subset="Company Name", inplace=True)

In [12]:
df.value_counts()

Company Name             Role                                                                         About The Job                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [13]:
df.drop_duplicates(subset="About The Job", inplace=True)

In [14]:
df.value_counts()

Company Name           Role                                                      About The Job                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [15]:
df.isnull().sum()

Company Name     0
Role             0
About The Job    0
dtype: int64

# Translate the Vietnamese text to english text

In [16]:
tqdm.pandas()
df["About The Job"] = df["About The Job"].progress_apply(translate_text)

  7%|▋         | 66/885 [00:02<01:00, 13.61it/s]

Translation error: the JSON object must be str, bytes or bytearray, not NoneType


 45%|████▍     | 398/885 [00:25<00:28, 17.00it/s]

Translation error: the JSON object must be str, bytes or bytearray, not NoneType


 51%|█████     | 453/885 [00:28<00:22, 19.49it/s]

Translation error: the JSON object must be str, bytes or bytearray, not NoneType


100%|██████████| 885/885 [00:50<00:00, 17.56it/s]


In [17]:
df["About The Job"] = df["About The Job"].apply(clean_text)

In [18]:
df.shape

(885, 3)

# Lower the text

In [19]:
df['About The Job'] = df['About The Job'].apply(lambda x: x.lower() if isinstance(x, str) else x)

# Tokenizer

In [20]:
tokens = [word_tokenize(c) for c in tqdm(df["About The Job"])]

100%|██████████| 885/885 [00:02<00:00, 334.12it/s]


In [21]:
len(tokens)

885

# Remove stopword

In [22]:
sw = stopwords.words("english")
tokens = [[word for word in t if word not in sw] for t in tokens]

In [23]:
len(tokens)

885

# Remove punctation

In [24]:
tokenizer = RegexpTokenizer(r'\w+')
tokens = [[" ".join(tokenizer.tokenize(word)) for word in t 
           if len(tokenizer.tokenize(word))>0] for t in tqdm(tokens)]

100%|██████████| 885/885 [00:00<00:00, 1784.39it/s]


In [25]:
len(tokens)

885

# Join the token become the text

In [26]:
df["About The Job"] = tokens

In [27]:
df["About The Job"] = [' '.join(token) for token in df["About The Job"]]

In [28]:
# # Khởi tạo biến đếm token
# total_tokens = 0

# # Lặp qua từng cột trong DataFrame
# for col in tqdm(df.columns):
#     # Lặp qua từng dòng văn bản trong cột và cộng số lượng token vào biến đếm
#     for text in df[col]:
#         if isinstance(text, str):  # Chỉ xử lý nếu dữ liệu là chuỗi
#             tokens = word_tokenize(text)  # Tokenize văn bản
#             total_tokens += len(tokens)  # Cộng số lượng token vào biến đếm

# # In ra tổng số token
# print(f"Tổng số token trong file CSV là: {total_tokens}")
total_tokens = 0

# Iterate over the "About The Job" column
for text in tqdm(df['About The Job']):
    if isinstance(text, str):  # Process only if the data is a string
        tokens = word_tokenize(text)  # Tokenize the text
        total_tokens += len(tokens)  # Add the number of tokens to the counter

# Print the total number of tokens
print(f"Total number of tokens in the 'About The Job' column: {total_tokens}")

100%|██████████| 885/885 [00:00<00:00, 885.94it/s] 

Total number of tokens in the 'About The Job' column: 265006


# Main

In [29]:
# Load spaCy model
nlp = spacy.load('en_core_web_sm')

if __name__ == '__main__':
    # Initialize matcher
    matcher = init_parser_func(nlp, SKILL_FILE_PATH, file_type="excel")
    
    # Initialize an empty list to collect data
    data_list = []
    
    # Iterate through each row in the DataFrame
    for c, (index, row) in enumerate(tqdm(df.iterrows(), total=len(df))):
        text = row['About The Job']
        role = row['Role']
        application_subject = row['Company Name']
        data = create_profile(nlp, matcher, text, role, application_subject)
        data_list.append(data)

    # Concatenate all collected data into final_database
    final_database = pd.concat(data_list, ignore_index=True)
    
    # Saving the database
    final_database.to_csv('./output/Data.csv', index=False)

    other_df = pd.DataFrame(final_database[['Company/Candidate Name', 'Skill', 'Sub-skill']])
    other_df.to_csv('./output/Skills.csv', index=False)


100%|██████████| 885/885 [01:07<00:00, 13.16it/s]


In [30]:
# plot_df(final_database)

# Group by Company name and Role

In [31]:
df_data = pd.read_csv("./output/Data.csv")
grouped_df = df_data.groupby(['Company/Candidate Name','Role']).agg({
    'Sub-skill': lambda x: ', '.join(x),
    'Count': 'sum'
}).reset_index()

# Print the grouped DataFrame
grouped_df.to_csv("./output/Filtered_Skill.csv")

In [42]:
# filtered_df = df[df["Company Name"] == "Jabil" & df["Role"]=="Financial Analyst II"]
filtered_df = df[(df["Company Name"] == "ABeam Consulting Vietnam") & (df["Role"] == "Business Development Manager (Korean)")]
filtered_df
# Print each job description for the filtered rows

for job_description in filtered_df["About The Job"]:
    print(wrap_text(job_description))

abeam consulting vietnamestablished june abeam vietnam consultants including japanese vietnamese experts digital transformation consulting
consolidated abeam consulting ltd japan abeam group people abeam consulting present countries offices global consulting technology outsourcing
services group reported global revenues jpy billion people expertise broad set industries service lines working clients organization team together
drive tangible results providing superb services every phase project learn us https www abeam com vn enjob overviewthis role mainly focus business
development consulting services korean companies vietnam main responsibilities develop implement strategic plans expand client s network korean
companies vietnamidentify new business opportunities analyzing market trends customer needs competitive landscape establish maintain strong
relationships key clients industry stakeholders business partners vietnam work closely bod align strategies ensure consistent brand messagin

In [33]:
df["Company Name"].value_counts()

Company Name
KTI Talent Indicator                 30
Publicis Groupe Vietnam              20
RMIT University                      17
AnyMind Group                        17
DKSH                                 13
                                     ..
Terumo Asia Pacific                   1
Funding Societies | Modalku Group     1
Astellas Pharma                       1
Sony Music Entertainment              1
Carousell Group                       1
Name: count, Length: 409, dtype: int64

# Dataframe Grouped

In [34]:
grouped_df.drop(columns="Count")

,Company/Candidate Name,Role,Sub-skill
0,2C2P,Partnerships Manager,"market research, business development, marketi..."
1,ABeam Consulting Vietnam,Business Development Manager (Korean),"vietnamese, digital, business development, cli..."
2,ABeam Consulting Vietnam,SAP Service Delivery Manager,"vietnamese, digital, optimization, statistics,..."
3,ABeam Consulting Vietnam,SAP SuccessFactors Consultant/Senior Consultant,"vietnamese, digital, client, problem solving, ..."
4,ACB - Asia Commercial Bank,Data Lake Developer,"big data, hadoop, business strategy, predictiv..."
...,...,...,...
856,dentsu,Digital Manager,"digital, client, project management, communica..."
857,dentsu,Media Intern,"digital, digital marketing, marketing, english..."
858,dentsu,Senior Creative Director,"creative, ownership, client, project managemen..."
859,interpool HR,IT Security Lead (m/f/d),"client, leadership, digital, ips, incident res..."
